In [1]:
import numpy as np 
import math


'''

obj_cam_space = [x_c, y_c, z_c, 1]

c2w = [[rf_11, rf_12, rf_13, rf_14],
       [rf_21, rf_22, rf_23, rf_24],
       [rf_31, rf_32, rf_33, rf_34],
       [rf_41, rf_42, rf_43, rf_44],]

obj_world_space = [x_w, y_w, z_w, 1] = c2w * obj_cam_space

x_w = rf_11*x_c + rf_12*y_c + rf_13*z_c + rf_14*1
y_w = rf_21*x_c + rf_22*y_c + rf_23*z_c + rf_14*1
z_w = rf_31*x_c + rf_32*y_c + rf_33*z_c + rf_14*1


if X ~ N[mu_x, sigma_x^2] then:
    (a*X + b) ~ N[a*mu_x+b, (a*sigma_x)^2]

if X ~ N[mu_x, sigma_x^2] and Y ~ N[mu_y, sigma_y^2] and Z ~N[mu_z, sigma_z^2] are independent, then:
    X+Y+Z ~ N[mu_x+mu_y+mu_z, sigma_x^2+sigma_y^2+sigma_z^2]

x_w ~ N[rf_11*mu_x_c + rf_12*mu_y_c + rf_13*mu_z_c + rf_14, (rf_11*sigma_x_c)^2 +(rf_12*sigma_y_c)^2 + (rf_13*sigma_z_c)^2]
y_w ~ N[rf_21*mu_x_c + rf_22*mu_y_c + rf_23*mu_z_c + rf_24, (rf_21*sigma_x_c)^2 +(rf_22*sigma_y_c)^2 + (rf_23*sigma_z_c)^2]
z_w ~ N[rf_31*mu_x_c + rf_32*mu_y_c + rf_33*mu_z_c + rf_34, (rf_31*sigma_x_c)^2 +(rf_32*sigma_y_c)^2 + (rf_33*sigma_z_c)^2]

'''

In [2]:
def get_transformatrix(cx, cy, cz, alpha, beta, gamma):
    
    # Transformation matrices (translation + rotations around x, y, z)
    mat_tran = np.array([[1,0,0,cx],
                         [0,1,0,cy],
                         [0,0,1,cz],
                         [0,0,0,1]])

    mat_rotx = np.array([[1,0,0,0],
                         [0,math.cos(alpha), -math.sin(alpha),0],
                         [0, math.sin(alpha), math.cos(alpha),0],
                         [0,0,0,1]])

    mat_roty = np.array([[math.cos(beta), 0, math.sin(beta),0],
                         [0,1,0,0],
                         [-math.sin(beta), 0, math.cos(beta),0],
                         [0,0,0,1]])


    mat_rotz = np.array([[math.cos(gamma), -math.sin(gamma), 0, 0],
                         [math.sin(gamma), math.cos(gamma),0, 0],
                         [0,0,1,0],
                         [0,0,0,1]])
    
    # General transformation matrix 'camera to world' (c2w)
    c2w = mat_tran.dot(mat_rotx).dot(mat_roty).dot(mat_rotz)
    
    return c2w

def get_world_gaussian(mu_x_c, mu_y_c, mu_z_c, sigma_x_c, sigma_y_c, sigma_z_c, c2w):
    
    mu_x_w = c2w[0,0]*mu_x_c + c2w[0,1]*mu_y_c + c2w[0,2]*mu_z_c + c2w[0,3]
    mu_y_w = c2w[1,0]*mu_x_c + c2w[1,1]*mu_y_c + c2w[1,2]*mu_z_c + c2w[1,3]
    mu_z_w = c2w[2,0]*mu_x_c + c2w[2,1]*mu_y_c + c2w[2,2]*mu_z_c + c2w[2,3]
    
    sigma_x_w = math.sqrt((c2w[0,0]*sigma_x_c)**2 +(c2w[0,1]*sigma_y_c)**2 + (c2w[0,2]*sigma_z_c)**2)
    sigma_y_w = math.sqrt((c2w[1,0]*sigma_x_c)**2 +(c2w[1,1]*sigma_y_c)**2 + (c2w[1,2]*sigma_z_c)**2)
    sigma_z_w = math.sqrt((c2w[2,0]*sigma_x_c)**2 +(c2w[2,1]*sigma_y_c)**2 + (c2w[2,2]*sigma_z_c)**2)
    
    return mu_x_w, mu_y_w, mu_z_w, sigma_x_w, sigma_y_w, sigma_z_w
  



In [6]:


# Camera Pose (in world coordinates with euler angles)
cx = 1
cy = 3
cz = -2
alpha = (math.pi/180)*(135)
beta = (math.pi/180)*(15)
gamma = (math.pi/180)*(-30)

# Get matrix to convert observations from camera space to world space
c2w = get_transformatrix(cx, cy, cz, alpha, beta, gamma)


obj_cam_space = [1, 1, 0 , 1]
obj_world_space = c2w.dot(obj_cam_space)

print("Camera Space:")
print(obj_cam_space)

print("World Space:")
print(obj_world_space)


mu_x_c = 0.8
mu_y_c = 0.3
mu_z_c = 0.5
sigma_x_c = 0.05
sigma_y_c = 0.03
sigma_z_c = 0.1


mu_x_w, mu_y_w, mu_z_w, sigma_x_w, sigma_y_w, sigma_z_w = get_world_gaussian(mu_x_c, mu_y_c, mu_z_c, sigma_x_c, sigma_y_c, sigma_z_c, c2w)

print(mu_x_w)
print(mu_y_w)
print(mu_z_w)
print(sigma_x_w)
print(sigma_y_w)
print(sigma_z_w)


Camera Space:
[1, 1, 0, 1]
World Space:
[ 2.31947922  2.99118095 -1.49118095  1.        ]
1.9435114394848672
2.911871455346716
-2.286390508185045
0.05127572188718037
0.07459701915564641
0.07215320559781566
